In [10]:
import os
os.environ["OMP_NUM_THREADS"] = "4"
import random
import gc
import re
import math
import hashlib
import datetime
import warnings
from collections import defaultdict
from multiprocessing import Pool, cpu_count

import mroc
import tensorflow
import pandas as pd
import lightgbm as lgb
import numpy as np
import keras
import pyarrow.parquet as parquet
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim import matutils
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.spatial.distance
from scipy import sparse
import nltk
from nltk.cluster import KMeansClusterer
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from pandas.util import hash_pandas_object
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

# Where the downloaded data are
input_path = '/Users/tyamgin/Projects/mlbootcamp/championship19/data'
# Where to store results
output_path = '/Users/tyamgin/Projects/mlbootcamp/championship19/res'

In [11]:
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.max_columns', 250)

In [12]:
cpu_count()

8

In [13]:
def hash_pandas_object2d(df):
    x = hash_pandas_object(df).values
    return hashlib.sha1(x).hexdigest()

In [14]:

keras_jobs = 8
config = tensorflow.ConfigProto(intra_op_parallelism_threads=keras_jobs,
                        inter_op_parallelism_threads=keras_jobs,
                        allow_soft_placement=True,
                        device_count = {'CPU': 8, 'GPU': 0})
session = tensorflow.Session(config=config)
keras.backend.set_session(session)

In [15]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    #torch.manual_seed(seed)
    #torch.cuda.manual_seed(seed)
    #torch.backends.cudnn.deterministic = True
    tensorflow.set_random_seed(seed)

In [23]:
a = pd.DataFrame({'a':[1,2,3],'b':[3,4,5]})
b = pd.DataFrame({'b':[3],'c':[222]})
a.join(b.set_index('b'), how='left', on='b')

,a,b,c
0,1,3,222.0
1,2,4,NaN
2,3,5,NaN


In [24]:
a.iloc[[0,2],:]

,a,b
0,1,3
2,3,5


In [20]:
pd.read_pickle(output_path + '/train_data').iloc[10000:20000,:]

,instanceId_userId,instanceId_objectType,objectId,audit_clientType,audit_timestamp,metadata_ownerId,metadata_createdAt,date,liked,clicked,viewed,disliked,reshared,ignored
10000,3425311,0,15340860,0,1517455724842,2800,1517393966000,2018-02-01,0,0,0,0,0,1
10001,3425821,0,27675711,0,1517476633442,13009,1517475615000,2018-02-01,0,0,0,0,0,1
10002,3425953,1,853025,1,1517504981081,11222,1517501910000,2018-02-01,0,0,0,0,0,1
10003,3426055,0,20839337,0,1517455210551,40120,1517371891000,2018-02-01,0,0,0,0,0,1
10004,3426055,0,28026133,0,1517456262180,71613,1517240131000,2018-02-01,0,0,0,0,0,1
10005,3426055,0,28026133,0,1517456262180,71613,1517240131000,2018-02-01,0,0,0,0,0,1
10006,3426055,0,33293164,0,1517488296983,21399,1517459927000,2018-02-01,0,0,0,0,0,1
10007,3426376,0,25347056,2,1517501306332,18976,1517434203000,2018-02-01,0,0,0,0,0,1
10008,3427354,0,25045890,1,1517482289920,26972,1517422224000,2018-02-01,1,0,0,0,0,0
10009,3427447,0,12320243,1,1517509448725,24741,1517466019000,2018-02-01,0,0,0,0,0,1


In [22]:
np.arange(5)

array([0, 1, 2, 3, 4])